In [16]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [17]:
%pip install neo4j
%pip install json-repair
%pip install py2neo
%pip install tree_sitter

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [18]:
%pip install --upgrade langchain
%pip install llama-index --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip show llama-index

Name: llama-index
Version: 0.11.13
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-legacy, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [20]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

from dotenv import load_dotenv

load_dotenv()


True

In [21]:
os.environ["NEO4J_URI"] 
os.environ["NEO4J_USERNAME"]
os.environ["NEO4J_PASSWORD"]

# Agora você pode criar o gráfico sem passar os parâmetros diretamente
graph = Neo4jGraph()

In [22]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Lista de caminhos de arquivos
file_paths = [
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoApiTokens.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoGrupo.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoGrupoUsuario.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoMenu.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoModuloTela.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoPermissao.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoPermissaoMenu.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoRecurso.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoTela.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoUsuario.sql",
    "/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoUsuarioCliente.sql"
]

# Carregar os documentos
docs = []
for file_path in file_paths:
    loader = TextLoader(file_path, encoding="utf-8")
    docs.extend(loader.load())  # Adicionar documentos carregados à lista

# Dividir os documentos em chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

# Verificar os documentos divididos
print(f"Total de documentos divididos: {len(documents)}")

Total de documentos divididos: 31


In [25]:
documents[0]

Document(metadata={'source': '/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoApiTokens.sql'}, page_content='\ufeffCREATE TABLE [dbo].[ControleDeAcessoApiTokens] (\n    [ID]                      INT           IDENTITY (1, 1) NOT NULL,\n    [ID_Empresa]              INT           NOT NULL,\n    [ID_Usuario]              INT           NOT NULL,\n    [Token]                   VARCHAR (500) NOT NULL,\n    [TokenExpiration]         DATETIME      NULL,\n    [DataDaUltimaAlteracao]   DATETIME      NOT NULL,\n    [UltimoUsuarioQueAlterou] INT           NOT NULL,')

In [26]:
# Obter as chaves da API
openai_api_key = os.getenv("OPENAI_API_KEY")
llama_api_key = os.getenv("LLAMA_API_KEY")

# Definir qual LLM será utilizado
llm_type = os.getenv("LLM_TYPE", "openai")  # Valor padrão "openai" se não estiver definido

if llm_type == "ollama":
    # Usar o modelo LLaMA/Ollama e passar a chave de API da LLaMA
    llm = ChatOllama(model="llama3.1", temperature=0, api_key=llama_api_key)
else:
    # Usar o modelo OpenAI e passar a chave de API da OpenAI
    llm = ChatOpenAI(temperature=0, model="gpt-4o", api_key=openai_api_key)

# Usar o modelo LLM no transformador de grafos
llm_transformer = LLMGraphTransformer(llm=llm)

# Converter documentos para o formato de grafo
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [27]:
graph_documents[0]

GraphDocument(nodes=[Node(id='Controledeacessoapitokens', type='Table', properties={}), Node(id='Id', type='Column', properties={}), Node(id='Id_Empresa', type='Column', properties={}), Node(id='Id_Usuario', type='Column', properties={}), Node(id='Token', type='Column', properties={}), Node(id='Tokenexpiration', type='Column', properties={}), Node(id='Datadaultimaalteracao', type='Column', properties={}), Node(id='Ultimousuarioquealterou', type='Column', properties={})], relationships=[Relationship(source=Node(id='Controledeacessoapitokens', type='Table', properties={}), target=Node(id='Id', type='Column', properties={}), type='HAS_COLUMN', properties={}), Relationship(source=Node(id='Controledeacessoapitokens', type='Table', properties={}), target=Node(id='Id_Empresa', type='Column', properties={}), type='HAS_COLUMN', properties={}), Relationship(source=Node(id='Controledeacessoapitokens', type='Table', properties={}), target=Node(id='Id_Usuario', type='Column', properties={}), type='

In [28]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [29]:
def showGraph():
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    
    session = driver.session()
    widget = GraphWidget(graph=session.run("MATCH (s)-[r:MENTIONS]->(t) RETURN s,r,t").graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [30]:
graph.query("SHOW INDEXES")

[{'id': 3,
  'name': 'constraint_907a464e',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'RANGE',
  'entityType': 'NODE',
  'labelsOrTypes': ['__Entity__'],
  'properties': ['id'],
  'indexProvider': 'range-1.0',
  'owningConstraint': 'constraint_907a464e',
  'lastRead': neo4j.time.DateTime(2024, 9, 25, 21, 37, 28, 170000000, tzinfo=<UTC>),
  'readCount': 2009},
 {'id': 1,
  'name': 'index_343aff4e',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'NODE',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 9, 25, 21, 38, 20, 441000000, tzinfo=<UTC>),
  'readCount': 666},
 {'id': 2,
  'name': 'index_f7700477',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'RELATIONSHIP',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRe

In [31]:
# Cria uma vectorstore
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

In [32]:
# Definir o modelo de entidades SQL para extrair informações
class SQLEntities(BaseModel):
    """Modelo para capturar entidades SQL como tabelas, colunas e relacionamentos."""
    tables: list[str] = Field(
        ...,
        description="Todas as tabelas presentes no texto SQL.",
    )
    
    columns: list[str] = Field(
        ...,
        description="Todas as colunas presentes no texto SQL.",
    )
    
    relationships: list[str] = Field(
        ...,
        description="Todos os relacionamentos (chaves estrangeiras, etc.) no texto SQL.",
    )

# Definir o prompt para extrair entidades SQL
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você está extraindo entidades SQL, como tabelas, colunas e relacionamentos, do texto SQL.",
        ),
        (
            "human",
            "Use o formato a seguir para extrair as informações do seguinte texto SQL."
            "input: {question}",
        ),
    ]
)

# Configurar o modelo LLaMA
llm = ChatOpenAI(model="gpt-4o", temperature=0)
entity_chain = llm.with_structured_output(SQLEntities)

In [33]:
entity_chain.invoke("O que é a tabela ControleDeAcessoUsuario?")

SQLEntities(tables=['ControleDeAcessoUsuario'], columns=[], relationships=[])